In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle

In [2]:
train_df_raw = pd.read_csv("./data/training.csv")

In [3]:
routes_df = pd.read_csv("./data/routes.csv")
route_list = routes_df['BUSROUTE_ID'].unique()

In [4]:
stop_df = pd.read_csv("./data/stops.csv")
stop_list = stop_df['BUSSTOP_ID'].unique()

In [5]:
bus_list = train_df_raw['BUS_ID'].unique()

In [6]:
train_df = train_df_raw

In [7]:
train_df.head()

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP
0,0,2020-01-06 07:00:00,325819008,11100360,388,21,1578294000
1,1,2020-01-06 07:00:00,451509115,11100480,112,9,1578294000
2,2,2020-01-06 07:00:00,235501055,11100500,374,9,1578294000
3,3,2020-01-06 07:00:00,145010001,11100330,331,30,1578294000
4,4,2020-01-06 07:00:01,421733004,11100620,51,29,1578294001


In [8]:
train_df = train_df.sort_values(by=['BUSROUTE_ID', 'BUS_ID', 'TIMESTAMP'])

In [9]:
route_index = 0
prev_seq = 0
seq_data = []
for idx, row in train_df.iterrows():
    if prev_seq == 0:
        prev_bus = row['BUS_ID']
        prev_seq = row['BUSSTOP_SEQ']
        prev_route = row['BUSROUTE_ID']
        prev_timestamp = row['TIMESTAMP']
        min_timestamp = row['TIMESTAMP']
        cnt = 1
        continue
        
    if (row['BUSSTOP_SEQ']>prev_seq and row['BUS_ID'] == prev_bus and row['BUSROUTE_ID'] == prev_route):
        pass
    else:
        seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])
        min_timestamp = row['TIMESTAMP']
        route_index+=1
        cnt = 0

        
        
    prev_bus = row['BUS_ID']
    prev_seq = row['BUSSTOP_SEQ']
    prev_route = row['BUSROUTE_ID']
    prev_timestamp = row['TIMESTAMP']
    train_df.at[idx, 'ROUTE_IDX'] = route_index
    cnt += 1
    
seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])

In [10]:
train_df=train_df.fillna(0)
for idx, row in train_df.iterrows():
    train_df.at[idx, 'ROUTE_TIME'] = seq_data[int(row['ROUTE_IDX'])][3]
    train_df.at[idx, 'SEQ_NUM'] = seq_data[int(row['ROUTE_IDX'])][2]

In [11]:
train_df['TIMESTAMP_DIFF'] = train_df['TIMESTAMP'].shift(-1) - train_df['TIMESTAMP']

In [12]:
train_df['SEQ_DIFF'] = train_df['BUSSTOP_SEQ'].shift(-1) - train_df['BUSSTOP_SEQ']

In [13]:
train_df['ROUTE_DIFF'] = train_df['BUSROUTE_ID'].shift(-1) - train_df['BUSROUTE_ID']

In [14]:
train_df['BUS_ID_DIFF'] = train_df['BUS_ID'].shift(-1) - train_df['BUS_ID']

In [15]:
# train_df = train_df[(train_df['SEQ_DIFF']==1.0) & (train_df['ROUTE_DIFF']==0.0) & (train_df['BUS_ID_DIFF']==0.0)]

In [16]:
train_df_bk = train_df

In [17]:
# train_df = train_df[train_df['SEQ_DIFF']==1]
train_df = train_df[train_df['SEQ_DIFF']>0]
train_df = train_df[train_df['ROUTE_DIFF']==0]
train_df = train_df[train_df['BUS_ID_DIFF']==0]

In [18]:
train_df[train_df['TIMESTAMP_DIFF']<1]
# train_df[train_df['SEQ_DIFF']<0]

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF


In [19]:
# bus id encoding
bus_id_encoder = preprocessing.LabelEncoder()
bus_id_encoder.fit(bus_list)
train_df.BUS_ID = bus_id_encoder.transform(train_df.BUS_ID)

# bus_id_encoder.transform([145010001, 235501055])
# bus_id_encoder.inverse_transform([280, 466])

In [20]:
# route id encoding
route_id_encoder = preprocessing.LabelEncoder()
route_id_encoder.fit(route_list)
train_df.BUSROUTE_ID = route_id_encoder.transform(train_df.BUSROUTE_ID)

In [21]:
# stop id encoding
stop_id_encoder = preprocessing.LabelEncoder()
stop_id_encoder.fit(stop_list)
train_df.BUSSTOP_ID = stop_id_encoder.transform(train_df.BUSSTOP_ID)

In [22]:
pickle.dump(bus_id_encoder  , open("./out/bus_id_encoder.pickle"    , "wb"))
pickle.dump(route_id_encoder, open("./out/route_id_encoder.pickle"  , "wb"))
pickle.dump(stop_id_encoder , open("./out/busstop_id_encoder.pickle", "wb"))

In [23]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,1.0,0.817500,21.0,117.0,1.0,0.0,0.0
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,1.0,0.817500,21.0,123.0,1.0,0.0,0.0
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,1.0,0.817500,21.0,121.0,1.0,0.0,0.0
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,1.0,0.817500,21.0,136.0,1.0,0.0,0.0
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,1.0,0.817500,21.0,174.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614746,1614746,2020-01-12 22:27:03,995,271,10,9,1578868023,89583.0,0.903056,13.0,123.0,1.0,0.0,0.0
1614825,1614825,2020-01-12 22:29:06,995,271,8,10,1578868146,89583.0,0.903056,13.0,185.0,1.0,0.0,0.0
1614944,1614944,2020-01-12 22:32:11,995,271,5,11,1578868331,89583.0,0.903056,13.0,99.0,1.0,0.0,0.0
1614996,1614996,2020-01-12 22:33:50,995,271,3,12,1578868430,89583.0,0.903056,13.0,225.0,1.0,0.0,0.0


In [24]:
train_df['DAY_OF_WEEK'     ] = pd.to_datetime(train_df['RECORD_DATE']).dt.dayofweek
train_df['HOUR_OF_DAY'     ] = pd.to_datetime(train_df['RECORD_DATE']).dt.hour
train_df['MINUTE_OF_HOUR'  ] = pd.to_datetime(train_df['RECORD_DATE']).dt.minute
train_df['SECOND_OF_MINUTE'] = pd.to_datetime(train_df['RECORD_DATE']).dt.second

In [25]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,1.0,0.817500,21.0,117.0,1.0,0.0,0.0,0,7,28,25
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,1.0,0.817500,21.0,123.0,1.0,0.0,0.0,0,7,30,22
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,1.0,0.817500,21.0,121.0,1.0,0.0,0.0,0,7,32,25
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,1.0,0.817500,21.0,136.0,1.0,0.0,0.0,0,7,34,26
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,1.0,0.817500,21.0,174.0,1.0,0.0,0.0,0,7,36,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614746,1614746,2020-01-12 22:27:03,995,271,10,9,1578868023,89583.0,0.903056,13.0,123.0,1.0,0.0,0.0,6,22,27,3
1614825,1614825,2020-01-12 22:29:06,995,271,8,10,1578868146,89583.0,0.903056,13.0,185.0,1.0,0.0,0.0,6,22,29,6
1614944,1614944,2020-01-12 22:32:11,995,271,5,11,1578868331,89583.0,0.903056,13.0,99.0,1.0,0.0,0.0,6,22,32,11
1614996,1614996,2020-01-12 22:33:50,995,271,3,12,1578868430,89583.0,0.903056,13.0,225.0,1.0,0.0,0.0,6,22,33,50


In [26]:
def timestamp_to_timeposition(time, starting_time = 21600): 
    if time < starting_time:
        return 0
    return (time - starting_time) % 86400

In [27]:
def timestamp_to_dayofweek(time):
    return (time//3600//24 + 3) % 7

In [28]:
timestamp_to_timeposition(1578607757)

58157

In [29]:
# time validation (valid time is 06:00:00 ~ 23:59:59)
starting_time = 21600 # 06:00:00
train_df['TIME_validation'] = train_df['TIMESTAMP'] % 86400
train_df = train_df[train_df['TIME_validation']>=starting_time]

In [30]:
# encodeing time

# (pd.to_datetime(train_df['RECORD_DATE']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# (pd.Timestamp('2020-01-14 07:00:00')  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train_df['TIME'] = (train_df['TIMESTAMP'] - starting_time) % 86400


c:\users\sharavsambuu\documents\github\kaggle-datanomads-bus-arrival\env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [31]:
train_df.sort_values(by=['TIMESTAMP_DIFF'])

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_validation,TIME
2495182,2495182,2020-01-16 13:20:24,610,206,378,16,1579180824,74618.0,1.865833,29.0,3.0,1.0,0.0,0.0,3,13,20,24,48024,26424
80314,80314,2020-01-06 11:18:34,725,251,379,27,1578309514,86647.0,2.068611,27.0,4.0,1.0,0.0,0.0,0,11,18,34,40714,19114
257759,257759,2020-01-07 07:23:00,296,260,1170,47,1578381780,88163.0,0.001111,2.0,4.0,1.0,0.0,0.0,1,7,23,0,26580,4980
1170937,1170937,2020-01-10 14:41:21,427,41,452,40,1578667281,14090.0,0.921389,40.0,5.0,1.0,0.0,0.0,4,14,41,21,52881,31281
2543779,2543779,2020-01-16 16:05:54,200,251,379,27,1579190754,85611.0,2.066667,27.0,5.0,1.0,0.0,0.0,3,16,5,54,57954,36354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512006,512006,2020-01-07 22:28:32,461,83,286,19,1578436112,30837.0,261.523056,37.0,937486.0,1.0,0.0,0.0,1,22,28,32,80912,59312
246902,246902,2020-01-06 22:02:59,451,41,375,21,1578348179,14486.0,275.429167,42.0,986435.0,1.0,0.0,0.0,0,22,2,59,79379,57779
247387,247387,2020-01-06 22:06:20,348,260,19,24,1578348380,88215.0,275.613056,47.0,986864.0,1.0,0.0,0.0,0,22,6,20,79580,57980
510697,510697,2020-01-07 22:12:35,739,206,378,16,1578435155,74805.0,275.450000,29.0,987255.0,1.0,0.0,0.0,1,22,12,35,79955,58355


In [32]:
# 2000 sec-ees ihiig evdersen esvel tur zogsolt hiisen gej uzeed haslaa

# train_df[(train_df['TIMESTAMP_DIFF']>2000)].sort_values(by=['TIMESTAMP_DIFF'])
# train_df = train_df[(train_df['TIMESTAMP_DIFF']<2000)]

In [33]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_validation,TIME
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,1.0,0.817500,21.0,117.0,1.0,0.0,0.0,0,7,28,25,26905,5305
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,1.0,0.817500,21.0,123.0,1.0,0.0,0.0,0,7,30,22,27022,5422
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,1.0,0.817500,21.0,121.0,1.0,0.0,0.0,0,7,32,25,27145,5545
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,1.0,0.817500,21.0,136.0,1.0,0.0,0.0,0,7,34,26,27266,5666
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,1.0,0.817500,21.0,174.0,1.0,0.0,0.0,0,7,36,42,27402,5802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614746,1614746,2020-01-12 22:27:03,995,271,10,9,1578868023,89583.0,0.903056,13.0,123.0,1.0,0.0,0.0,6,22,27,3,80823,59223
1614825,1614825,2020-01-12 22:29:06,995,271,8,10,1578868146,89583.0,0.903056,13.0,185.0,1.0,0.0,0.0,6,22,29,6,80946,59346
1614944,1614944,2020-01-12 22:32:11,995,271,5,11,1578868331,89583.0,0.903056,13.0,99.0,1.0,0.0,0.0,6,22,32,11,81131,59531
1614996,1614996,2020-01-12 22:33:50,995,271,3,12,1578868430,89583.0,0.903056,13.0,225.0,1.0,0.0,0.0,6,22,33,50,81230,59630


In [34]:
output_df = train_df[[
    "DAY_OF_WEEK", "HOUR_OF_DAY", "MINUTE_OF_HOUR", "SECOND_OF_MINUTE", "TIME", 
    "BUS_ID", "BUSROUTE_ID", "BUSSTOP_ID", "TIMESTAMP_DIFF", "ROUTE_TIME", "SEQ_NUM"
]]

In [35]:
output_df.to_csv('./out/cleaned_train.csv', index = False)